# MNIST

아래의 code는 google colab 에서 시험된 결과이다.
Anaconda 에서 제공되는 jupyter notebook에서는 작동이 안될수 있다.
추천하는 방법은 notebook이 아니라 python 파일 자체를 실행하라는 것이다. TPU 예제만 빼고는 anaconda prompt 에서 *.py 파일을 실행하는 방법을 추천한다.
compare 도구로서는 "BCcompare"를 추천한다. working day로 30일간 무료로 사용이 가능하다.
무료 사용후 재 설치 하면 다시 30 working day동안 사용이 가능하다.

Compare 도구를 사용하여 "11_TF2_MNIST_sparse_network_modify.py"와 "21_TF2_MNIST_expert_sequential_sparse.py"를 비교하여 보면 "tf.data.Dataset.from_tensor_slices"가 추가가 되어진것을 볼수 있다.


```
batch_size = 1000
# 입력된 buffer_size만큼 data를 채우고 무작위로 sampling하여 새로운 data로 바꿉니다.
# 완벽한 셔플링을 위해서는 데이터 세트의 전체 크기보다 크거나 같은 버퍼 크기가 필요합니다.
# 만약 작은 데이터수보다 작은 buffer_size를 사용할경우,
# 처음에 설정된 buffer_size만큼의 data안에서 임의의 셔플링이 발생합니다.
shuffle_size = 100000

train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, Y_train)).shuffle(shuffle_size).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(batch_size)

```

"tf.data.Dataset."의 사용법에 대한 자세한 강의는 다른 부분에서 진행할 예정입니다.
이를 사용하는 주된 목적은 deel learning machine의 resource 때문으로 보면 됩니다.

Tensorflow에서 제공하는 MNIST나 cifar10, cifar100를 일반 laptop의 memory 에서 실행이 가능합니다.
하지만 image의 사이즈가 224 pixel로만 커져도 감당이 되지 않고, data를 쪼개서 학습을 시켜야 합니다.
이 저장소에서는 이러한것이 있다는 것 정도만 알면 됩니다.



In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras import Model, Sequential

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# print(tf.__version__)
## MNIST Dataset #########################################################
mnist = tf.keras.datasets.mnist
# class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
##########################################################################

## Fashion MNIST Dataset #################################################
# mnist = tf.keras.datasets.fashion_mnist
# class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
##########################################################################
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()    

# Change data type as float. If it is unt type, it might cause error 
X_train, X_test = X_train / 255.0, X_test / 255.0

print(Y_train[0:10])

print(X_train.shape)

# in the case of Keras or TF2, type shall be [image_size, image_size, 1]
# if it is RGB type, type shall be [image_size, image_size, 3]
# For MNIST or Fashion MNIST, it need to reshape
X_train = X_train[..., tf.newaxis]
X_test = X_test[..., tf.newaxis]

print(X_train.shape)

batch_size = 1000
# 입력된 buffer_size만큼 data를 채우고 무작위로 sampling하여 새로운 data로 바꿉니다.
# 완벽한 셔플링을 위해서는 데이터 세트의 전체 크기보다 크거나 같은 버퍼 크기가 필요합니다.
# 만약 작은 데이터수보다 작은 buffer_size를 사용할경우,
# 처음에 설정된 buffer_size만큼의 data안에서 임의의 셔플링이 발생합니다.
shuffle_size = 100000

train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, Y_train)).shuffle(shuffle_size).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(batch_size)


11493376/11490434 [==============================] - 0s 0us/step
[5 0 4 1 9 2 1 3 1 4]
(60000, 28, 28)
(60000, 28, 28, 1)


아래의 code 부분은 초보자 모드와 동일하다.
네트워크를 구성하는 방법은 바뀔필요는 없다.

In [2]:

model = Sequential([
    Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME',input_shape=(28, 28, 1)),
    MaxPool2D(padding='SAME'),
    Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME'),
    MaxPool2D(padding='SAME'),
    Conv2D(filters=256, kernel_size=3, activation=tf.nn.relu, padding='SAME'),
    MaxPool2D(padding='SAME'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 256)         295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 256)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0

Compare 도구를 사용하여 "11_TF2_MNIST_sparse_network_modify.py"와 "21_TF2_MNIST_expert_sequential_sparse.py"를 비교하여 보면, 전문가 모드에서는 "tf.GradientTape()"가 사용되져서 구성되어져 있다는것을 알수 있다.

아래의 코드가 복잡하게 보이는 이유는 "tf.data.Dataset."와 "tf.GradientTape()"이 동시에 사용되어 졌기 때문이다.

"tf.GradientTape()"가 필요한 시점이 오면 이를 자세히 다루도록 하겠다.
아래의 code를 향후에 복잡한 code들을 구성할때에 약간씩만 고쳐서 사용하도록 할 예정이므로, 아래를 통째로 가지고 다니도록 하자.

In [3]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = 'epoch: {:>5,d}, loss: {:>2.4f}, accuracy: {:>2.3f}%, test loss: {:>2.4f}, test accuracy: {:>2.3f}%'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))           


epoch:     1, loss: 0.5213, accuracy: 83.848%, test loss: 0.1058, test accuracy: 96.390%
epoch:     2, loss: 0.3012, accuracy: 90.673%, test loss: 0.0774, test accuracy: 97.420%
epoch:     3, loss: 0.2171, accuracy: 93.277%, test loss: 0.0623, test accuracy: 97.913%
epoch:     4, loss: 0.1722, accuracy: 94.661%, test loss: 0.0560, test accuracy: 98.145%
epoch:     5, loss: 0.1442, accuracy: 95.528%, test loss: 0.0510, test accuracy: 98.302%
